In [ ]:
# Автоперезагрузка импортов
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sys
import torch
import os

# Отключаем параллелизм в токенизаторах (иначе могут быть проблемы с многопроцессорностью)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Подключаем нужные пути
SRC_DIR = Path('src')
MODELS_DIR = Path('models')
sys.path.append(str(SRC_DIR))

from data_utils import create_dataset_processed, split_dataset
from next_token_dataset import create_loaders
from lstm_model import SimpleLSTMNextToken, count_parameters
from eval_lstm import eval_rouge, autocomplete_examples
from lstm_train import train_one_epoch, validate

# Очистка датасета и токенизация
n_rows = create_dataset_processed()
print(f"[OK] Создан очищенный датасет, строк: {n_rows}.")

# Разбиение на выборки train/val/test
n_train, n_val, n_test = split_dataset()
print(f"[OK] Созданы тренировочная, валидационная, тестовая выборки: {n_train}, Val: {n_val}, Test: {n_test}.")

# Создание Dataset и DataLoader
(dsets, loaders, tokenizer) = create_loaders(
    data_dir='data',
    tokenizer_name='bert-base-multilingual-cased',
    seq_len=32,
    batch_size=64,
    add_eos=True,
)

train_ds, val_ds, test_ds = dsets
train_loader, val_loader, test_loader = loaders
print(f"[OK] Созданы Dataset и DataLoader для обучения модели:")
print(f"Train batches: {len(train_loader)} | Val batches: {len(val_loader)}")

batch = next(iter(train_loader))
for k, v in batch.items():
    print(k, v.shape)

# Создание модели
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pad_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0
model = SimpleLSTMNextToken(
    vocab_size=tokenizer.vocab_size,
    emb_dim=256,
    hidden_dim=128,
    num_layers=1,
    pad_idx=pad_id,
    dropout=0.1,
).to(device)

print("[OK] Модель создана. Кол-во обучаемых параметры:", count_parameters(model))

criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)  # labels у нас уже -100 на паддингах
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-3, weight_decay=1e-2)

# === Тренировка: выводим train loss, val loss и ROUGE на вал. ===
from time import time

epochs = 2
log_every = 100
rouge_max_batches = 10   # увеличить для более точной оценки (медленнее)

best_val = float("inf")
best_ckpt_path = MODELS_DIR / "lstm_next_token_best.pt"

print(f"[OK] Начинаем тренировку на {epochs} эпох:")
for epoch in range(1, epochs + 1):
    print(f"\n=== Epoch {epoch}/{epochs} ===")
    t0 = time()

    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device, log_every=log_every)
    val_loss   = validate(model, val_loader, criterion, device)

    # ROUGE (3/4 → 1/4) на валидации
    rouge = eval_rouge(model, val_loader, tokenizer, pad_id=pad_id, max_batches=rouge_max_batches)

    dt = time() - t0
    print(f"Epoch {epoch}: train loss {train_loss:.4f} | val loss {val_loss:.4f} | time {dt:.1f}s")
    print("ROUGE:", " ".join([f"{k}={v:.4f}" for k, v in rouge.items()]))

    # Сохраняем чекпойнт за эпоху
    ckpt = {
        "state_dict": model.state_dict(),
        "hparams": {
            "emb_dim": 256,
            "hidden_dim": 128,
            "num_layers": 1,
            "dropout": 0.1,
            "seq_len": 32,
            "tokenizer_name": "bert-base-multilingual-cased",
        },
        "metrics": {
            "train_loss": float(train_loss),
            "val_loss": float(val_loss),
            **{f"rouge/{k}": float(v) for k, v in rouge.items()},
        },
        "epoch": epoch,
    }
    ep_path = MODELS_DIR / f"lstm_next_token_e{epoch}_valloss{val_loss:.3f}.pt"
    torch.save(ckpt, ep_path)
    print("Saved:", ep_path)

    if val_loss < best_val:
        best_val = val_loss
        torch.save(ckpt, best_ckpt_path)
        print("→ Updated best:", best_ckpt_path)

print(f"[OK] Модель обучена.")

# === Тестовая метрика ROUGE + примеры автодополнений ===
# Загружаем лучший чекпойнт (если нужно)
state = torch.load(best_ckpt_path, map_location="cpu")
model.load_state_dict(state["state_dict"])
model.to(device).eval()

# ROUGE на тесте
r_test = eval_rouge(model, test_loader, tokenizer, pad_id=pad_id, max_batches=rouge_max_batches)
print("[OK] ROUGE метрика:")
for k, v in r_test.items():
    print(f"  {k}: {v:.4f}")

# Несколько примеров (FULL / PREFIX / TARGET / PRED)
print("[OK] Примере автозаполнений:")
autocomplete_examples(model, test_loader, tokenizer, pad_id=pad_id, num_examples=5)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[OK] Создан очищенный датасет /Users/alexeypokrovsky/Documents/Work/Learning/Development/Backend/Python/ai-text-autocomplete/data/dataset_processed.csv, строк: 1600498
Processed rows: 1600498
[OK] Созданы тренировочная, валидационная, тестовая выборки: 1280398, Val: 160050, Test: 160050
Splits: 1280398 160050 160050
[OK] Созданы Dataset и DataLoader для обучения модели.
Train batches: 4165 | Val batches: 510
batch:
input_ids torch.Size([64, 32])
labels torch.Size([64, 32])
batch first tokens:
tensor([10216, 10237, 35432, 10107, 10911, 15311, 10833, 22591, 10114, 11783])
tensor([10237, 35432, 10107, 10911, 15311, 10833, 22591, 10114, 11783, 10950])
Trainable params: 46223227

=== Epoch 1/2 ===
  step   100 | loss 6.9302
  step   200 | loss 6.8629
  step   300 | loss 6.4701
  step   400 | loss 6.2009
  step   500 | loss 5.9676
  step   600 | loss 5.8187
  step   700 | loss 5.7798
  step   800 | loss 5